In [ ]:
%matplotlib inline

import numpy as np
from matplotlib import pyplot as plt
import time
import tt

def rs(f1, f2, vx_l, vx_r):
    return vx_l * f1 + vx_r * f2
    
def minmod(a, b):
    return np.where((a*b > 0), np.where(((abs(a) <= abs(b)) & (a*b > 0)), a, b), 0)    
    
def minmod2(a, b):
    if ((abs(a) <= abs(b)) & (a*b > 0)):
        return a
    elif (a*b > 0):
        return b
    else:
        return 0

def maxmod(a, b):
    return np.where((a*b > 0), np.where(((abs(a) >= abs(b)) & (a*b > 0)), a, b), 0)

def minmod_lim(u):
    return minmod(u[1:-1] - u[:-2], u[2:] - u[1:-1])

def superbee(u):
    return maxmod(minmod(u[2:] - u[1:-1], 2*(u[1:-1] - u[:-2])),
                             minmod(2*(u[2:] - u[1:-1]), u[1:-1] - u[:-2]))


def F_m(vx, vy, vz, T, n, p):
    return n * ((1. / (2. * np.pi * p.Rg * T)) ** (3. / 2.)) * (np.exp(-(vx*vx + vy*vy + vz*vz) / (2. * p.Rg * T)))

def J(f, vx, vy, vz, vx_tt, vy_tt, vz_tt, hv, N, r, p):
    
    n = (hv ** 3) * tt.sum(f)

    ux = (1. / n) * (hv ** 3) * tt.sum(vx_tt * f)
    uy = (1. / n) * (hv ** 3) * tt.sum(vy_tt * f)
    uz = (1. / n) * (hv ** 3) * tt.sum(vz_tt * f)
    
    v2 = (vx_tt*vx_tt + vy_tt*vy_tt + vz_tt*vz_tt).round(r)
    u2 = ux*ux + uy*uy + uz*uz
    
    T = (1. / (3. * n * p.Rg)) * ((hv ** 3) * tt.sum(v2 * f) - n * u2)

    Vx = vx - ux
    Vy = vy - uy
    Vz = vz - uz
    
    Vx_tt = (vx_tt - ux * tt.ones((N,N,N))).round(r)
    Vy_tt = (vy_tt - uy * tt.ones((N,N,N))).round(r)
    Vz_tt = (vz_tt - uz * tt.ones((N,N,N))).round(r)

    rho = p.m * n

    P = rho * p.Rg * T

    cx = Vx_tt * (1. / ((2. * p.Rg * T) ** (1. / 2.)))
    cy = Vy_tt * (1. / ((2. * p.Rg * T) ** (1. / 2.)))   
    cz = Vz_tt * (1. / ((2. * p.Rg * T) ** (1. / 2.)))
    
    c2 = (cx*cx + cy*cy + cz*cz).round(r)

    Sx = (1. / n) * (hv ** 3) * tt.sum(cx * c2 * f)
    Sy = (1. / n) * (hv ** 3) * tt.sum(cy * c2 * f)
    Sz = (1. / n) * (hv ** 3) * tt.sum(cz * c2 * f)

    mu = p.mu_0 * ((p.T_0 + p.C) / (T + p.C)) * ((T / p.T_0) ** (3. / 2.))

    F_M = tt.tensor(F_m(Vx, Vy, Vz, T, n, p))
    
    f_plus = F_M * (tt.ones((N,N,N)) + (4. / 5.) * (1. - p.Pr) * (cx*Sx + cy*Sy + cz*Sz) * (c2 - (5. / 2.) * tt.ones((N,N,N))))
    f_plus = f_plus.round(r)
    
    J = (f_plus - f) * (P / mu)
    J = J.round(r)
    
    return J, n, ux, T

def save_tt(filename, f, L, N):
    
    m = max(f[i].core.size for i in range(L))

    F = np.zeros((m+4, L))
    
    for i in range(L):
        F[:4, i] = f[i].r.ravel()
        F[4:f[i].core.size+4, i] = f[i].core.ravel()
    
        np.savetxt(filename, F)#, fmt='%s')
    
def load_tt(filename, L, N):
    
    F = np.loadtxt(filename)
    
    f = list()
    
    for i in range(L):
        
        f.append(tt.rand([N, N, N], 3, F[:4, i]))
        f[i].core = F[4:f[i].core.size+4, i]
        
    return f

In [ ]:
def solver(x_l, x_r, L, Tau, CFL, vmax, N, n_l, u_l, T_l, r, p):
    
    h = (x_r - x_l) / L 
    tau = h * CFL / vmax / 10
    
    x = np.linspace(x_l+h/2, x_r-h/2, L)
    
    t = 0.
    
    hv = 2. * vmax / N
    vx_ = np.linspace(-vmax+hv/2, vmax-hv/2, N)
    vx, vy, vz = np.meshgrid(vx_, vx_, vx_, indexing='ij')
    
    vx_l = np.zeros((N, N, N))
    vx_r = np.zeros((N, N, N))
    
    for i in np.ndindex(vx.shape):
        if vx[i] > 0:
            vx_l[i] = vx[i]
        else:
            vx_l[i] = 0.
            
    vx_r = vx - vx_l
            
    vx_l = tt.tensor(vx_l)
    vx_r = tt.tensor(vx_r)
    
    vx_tt = tt.tensor(vx)
    vy_tt = tt.tensor(vy)
    vz_tt = tt.tensor(vz)
    
    M = u_l / ((p.g * p.Rg * T_l) ** .5)
    
    n_r = (p.g + 1.) * M * M / ((p.g - 1.) * M * M + 2.) * n_l
    u_r = ((p.g - 1.) * M * M + 2.) / ((p.g + 1.) * M * M) * u_l
    T_r = (2. * p.g * M * M - (p.g - 1.)) * ((p.g - 1.) * M * M + 2.) / ((p.g + 1) ** 2 * M * M) * T_l
    
    
    F_l = tt.tensor(F_m(vx-u_l, vy, vz, T_l, n_l, p))
    F_r = tt.tensor(F_m(vx-u_r, vy, vz, T_r, n_r, p))
    
#    vx = tt.tensor(vx)
#    vy = tt.tensor(vy)
#    vz = tt.tensor(vz)
    
    # initial condition 
#    f = list(F_l for i in range(L/2+1))
#    f = f.extend(list(F_r for i in range(L/2+1, L)))

    f = list(tt.tensor(np.zeros((N, N, N))) for i in range(L))
    
    for i in range(L/2+1):
        f[i] = F_l
    for i in range(L/2+1, L):
        f[i] = F_r
        
#    for i in range(L):
#        f[i] = tt.tensor(f[i])
    
    slope = list(0. * tt.ones((N,N,N)) for i in range(L))
    tmp = list(0. * tt.ones((N,N,N)) for i in range(L))
    f_l = list(0. * tt.ones((N,N,N)) for i in range(L+1))
    f_r = list(0. * tt.ones((N,N,N)) for i in range(L+1))
    Flow = list(0. * tt.ones((N,N,N)) for i in range(L+1))
    RHS = list(0. * tt.ones((N,N,N)) for i in range(L))
    J_ = list(0. * tt.ones((N,N,N)) for i in range(L))
    
#    Frob_norm = np.array([])
    Frob_norm = np.zeros(L)
#    C_norm = np.array([])
        
    t1 = time.clock()
    
    while(t < Tau*tau):
        t += tau
        # boundary condition
        f_l[0] = F_l
        f_r[L] = F_r
        # reconstruction
        # compute slopes
        
        for i in range(1, L-1):
            slope[i] = h * (f[i+1] - 2 * f[i] + f[i-1])
            slope[i] = slope[i].round(r)
#            print slope[i] + f[i]
#            slope[i] = slope[i].round(rmax=150)
            # insert ROUNDING!!!
#            slope[i, :, :, :] = h * minmod(f[i+1, :, :, :] - f[i, :, :, :], f[i, :, :, :] - f[i-1, :, :, :])
            
        for i in range(L):
            f_r[i] = f[i] - slope[i] * (h / 2)
            f_r[i] = f_r[i].round(r)
        
        for i in range(1, L+1):
            f_l[i] = f[i-1] + (h / 2) * slope[i-1]
            f_l[i] = f_l[i].round(r)
        
        # riemann solver - compute fluxes
        for i in range(L+1):
            Flow[i] = rs(f_l[i], f_r[i], vx_l, vx_r)
            Flow[i] = Flow[i].round(r)
            
        # compute RHS
        for i in range(L):
            J_[i] = J(f[i], vx, vy, vz, vx_tt, vy_tt, vz_tt, hv, N, r, p)
            RHS[i] = (- Flow[i+1] + Flow[i]) * (1. / h) + J_[i][0]
            RHS[i] = RHS[i].round(r)
            
        if (((t/tau) % 100) == 1):
            for i in range(L):
                Frob_norm[i] = RHS[i].norm()
                
                
            fig, ax = plt.subplots(figsize = (20,10))
            ax.plot(Frob_norm)

            ax.set(title='RHS frob norm')

            fig.savefig("RHS.png")
            
            
            save_tt('file.txt', f, L, N)
                

#           C_norm = np.append(C_norm, np.max(np.absolute(np.ravel(RHS))))

        # update values
        for i in range(L):
            tmp[i] = f[i] + tau * RHS[i]
            tmp[i] = tmp[i].round(r)
            
        for i in range(L):    
            f[i] = tmp[i]
        
#        print np.linalg.norm(RHS)
#        print np.max(np.absolute((j)))
        
     
        
    t2 = time.clock() - t1
    
    Dens = np.zeros(L)
    Vel = np.zeros(L)
    Temp = np.zeros(L)



    for i in range(L):
        
        Dens[i] = J_[i][1]
        Vel[i] = J_[i][2]
        Temp[i] = J_[i][3]


    print "time =", t2
    
    
    return f, Dens, Vel, Temp#, Frob_norm#, C_norm

In [ ]:
class Params(object):
 
    def __init__(self):
        #fundamental constants
        self.Na = 6.02214129e+23
        self.kB = 1.381e-23 # J / K
        #gas parameters
        self.Mol = 40e-3 # kg / mol
        self.Rg = 8.3144598  / self.Mol  # J / (kg * K) 
        self.m = self.Mol / self.Na # kg
    
        self.Pr = 2. / 3.
        self.C = 144.4
        self.T_0 = 273.11
        self.mu_0 = 2.125e-05
    
        self.g = 5. / 3.
        
        self.d = 71e-12

p = Params()

L = 40
N = 20
vmax = 2000.

n_l = 2e+23 
u_l = 400.
T_l = 300.

M = u_l / ((p.g * p.Rg * T_l) ** .5)

n_r = (p.g + 1.) * M * M / ((p.g - 1.) * M * M + 2.) * n_l
u_r = ((p.g - 1.) * M * M + 2.) / ((p.g + 1.) * M * M) * u_l
T_r = (2. * p.g * M * M - (p.g - 1.)) * ((p.g - 1.) * M * M + 2.) / ((p.g + 1) ** 2 * M * M) * T_l


l = 1. / ((2 ** .5) * np.pi * n_l * p.d * p.d)

x_l = -40*l
x_r = 40*l

#x_l = -20*l
#x_r = 20*l

Tau = 15001

CFL = 0.5

r = 1e-7

S = solver(x_l = x_l, x_r = x_r, L = L, Tau = Tau, CFL = CFL, vmax = vmax,
           N = N, n_l = n_l, u_l = u_l, T_l = T_l, r = r, p = p)

In [ ]:
fig, ax = plt.subplots(figsize = (20,10))
line, = ax.plot(6 * (S[1] - n_l) / (n_r - n_l))#,'ko-')
line.set_label('Density')
line, = ax.plot(6 * (S[2] - u_r) / (u_l - u_r))
line.set_label('Velocity')
line, = ax.plot(6 * (S[3] - T_l) / (T_r - T_l))
line.set_label('Temperature')

s = np.zeros(L)
for i in range(L):
    s[i] = (S[0][i].r[1] + S[0][i].r[2]) / 2.
    
#s = (s - np.min(s)) / (np.max(s) - np.min(s))
    
line, = ax.plot(s,'ko-')
line.set_label('Mean Rank')

ax.legend()


#ax.set_xlim((30,60))

#print S[0]

#Frob_norm = 2.4855822240924352e+17

#ax.plot(F_l[:, N/2, N/2])
#ax.plot(F_r[:, N/2, N/2])
#ax.plot(Flow[:, N/2, N/2])

#print vx_

#ax.plot(F_r[:, 10, 10])
#print n_l, n_r
#print u_l, u_r
#print T_l, T_r

In [ ]:
fig, ax = plt.subplots(figsize = (20,10))
line, = ax.plot(S[4])
line.set_label('Frob_norm')
#line, = ax.semilogy(S[5])
#line.set_label('C_norm')
#ax.set_ylim((1e+16,1e+17))
ax.legend()

In [ ]:
d = 71e-12

Mol = 40e-3 # kg / mol
Rg = 8.3144598  / Mol  # J / (kg * K) 

n = 2e+23 
u = 400.
T = 300.

vx_l = np.zeros((N, N, N))
vx_r = np.zeros((N, N, N))
    
for i in np.ndindex(vx.shape):
    if vx[i] > 0:
        vx_l[i] = vx[i]
    else:
        vx_l[i] = 0.
            
vx_r = vx - vx_l
            
vx_l = tt.tensor(vx_l)
vx_r = tt.tensor(vx_r)

f = tt.tensor(np.zeros((N, N, N)))

f1 = tt.tensor(F_m(vx-100., vy, vz, T, n, Mol))
f2 = tt.tensor(F_m(vx+100., vy, vz, T, n, Mol))

f = rs(f1, f2, f, vx_l, vx_r)

print f

plt.plot(f.full()[:, 20, 20])

In [ ]:
slope = tt.tensor(np.zeros((10, 10, 10)))
print slope

In [ ]:
?tt.ones()

In [ ]:
f = S[0]
f

In [ ]:
plt.figure(figsize = (20,10))
g = lambda x: np.exp(-x**2)
xx = np.linspace(-5,4, 4096)
plt.plot(xx, g(xx))

shape = [2 for i in range(12)]
shape = tuple(shape)
    
U = np.reshape(g(xx), shape)

U_tt = tt.tensor(U,5e-2)
print(U_tt)
print(U_tt.core.size)
U_tt_full = U_tt.full()
U_tt_1d = np.ravel(U_tt_full)

plt.plot(xx,U_tt_1d)

In [ ]:
A = S[0][0]  # tensor object
np.savez('tensor.npz', *A.to_list(tt))  # store into tensor.npz
B = tt.tensor.from_list(np.load('tensor.npz').items())  # load from tensor.npz

In [ ]:
t = tt.ones((N,N,N))
t.core = A.core
t_f = t.full()
#print(t_f)
plt.plot(t_f[:,20,20])
tt.tensor.from_list

In [ ]:
f = S[0]
#plt.plot(f[19].full())
#print f[19].r
#f[19].r = [1, 1, 1, 1]
for i in range(L):
    print f[i].r, f[i].core.size

In [ ]:
#f_2 = tt.rand([10, 10, 10], 3, [1, 3, 3, 1])

#print f_2.core.size

In [ ]:
save_tt('file.txt', f, L, N)

In [ ]:
a = load_tt('file.txt', L, N)

In [ ]:
err = (f[10] - a[10]).full()
print err

In [ ]:
print np.linalg.norm(err)
dir(f[17])
print f[17].norm()

In [ ]:
class Param:
    pass
p = Param()
p.Mach = 2
p.R  = 8.31
print(p.Mach)

In [ ]:
a = load_tt('file.txt', L, N)
Dens = J(a)